In [3]:
import numpy as np
from functions import *
from gradient import numerical_gradient

In [23]:
class simpleNet:    #간단한 신경망 구현
    def init(self):
        self.W = np.random.randn(2, 3) #2x3 가중치 배열
    def predict(self, x):               #예측값 반환
        return np.dot(x, self.W)
    def loss(self, x, t):
        z = self.predict(x)         #z: 예측값
        y = softmax(z)              #y = a.f값
        loss = cross_entropy_error(y, t)    #교제 제곱 오차: 타깃과 예측의 차이
        return loss
x = np.array([0.6, 0.9])    #입력값
t = np.array([0, 0, 1]) #target: 정답
net = simpleNet()
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)
for i in range(5):
    print("\nWeight: \n", net.W)
    print("activated: \n", softmax(net.predict(x)))
    print("Loss: \n", net.loss(x, t))
    print("="*50)
    net.W -= dW


AttributeError: 'simpleNet' object has no attribute 'W'

In [43]:
class simpleNet:
    def __init__(self, a, b):   #a: X.r, b: X.c
        self.W = np.random.randn(a, b)
    def predict(self, x):
        return np.dot(x, self.W)
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss
    
x = np.array([0.5, -2.2, -7])
t = np.array([0, 1, 0, 0])
net = simpleNet(3, 4)
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)
for i in range(5):
    print("\nWeight: \n", net.W)
    print("activated: \n", softmax(net.predict(x)))
    #print("Loss: \n", net.loss(x, t))
    print("="*50)
    net.W -= dW


Weight: 
 [[ 0.17081106  0.00292529  1.95807241 -0.18515059]
 [ 0.53770961 -1.01906504 -1.34387893  0.03628067]
 [-0.60927881 -0.03967839 -0.58740372  0.07659958]]
activated: 
 [7.50889540e-03 3.93474772e-03 9.88400668e-01 1.55688578e-04]

Weight: 
 [[ 0.16705671  0.50094526  1.46388463 -0.18522843]
 [ 0.55422876 -3.21035291  0.83054727  0.03662317]
 [-0.55671788 -7.01195796  6.33122512  0.07768937]]
activated: 
 [5.08521712e-24 1.00000000e+00 6.08323673e-45 1.56935039e-25]

Weight: 
 [[  0.16330236   0.99896523   0.96969686  -0.18530627]
 [  0.57074791  -5.40164077   3.00497348   0.03696568]
 [ -0.50415694 -13.98423752  13.24985396   0.07877917]]
activated: 
 [1.35506410e-47 1.00000000e+00 1.47317146e-91 6.22443558e-49]

Weight: 
 [[  0.15954801   1.4969852    0.47550908  -0.18538411]
 [  0.58726706  -7.59292864   5.17939969   0.03730819]
 [ -0.45159601 -20.95651709  20.1684828    0.07986896]]
activated: 
 [3.61085606e-071 1.00000000e+000 3.56756484e-138 2.46876661e-072]

Weight: 
 [

In [44]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01): #입력 크기, 히든 레이어 크기, 출력 크기, 실수값으로 조지기
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) #입력x히든 사이즈로 난수 배열 만들고 실수로 값 변환
        self.params['b1'] = np.zeros(hidden_size)   #히든 레이어 개수만큼 
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        return y
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0]) #np.sum(y==t) y==t인 case의 횟수를 저장, 이를 평균을 내줌
        return accuracy                               #그렇게 구한 정확도를 반환
    
    def numerical_gradient(self, x, t):                 #손실함수를 w에 대해 미분함 -> gradient를 반환
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        batch_num = x.shape[0]
        #forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        #backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)
        return grads


In [51]:
from datasetsub.mnist import load_mnist
(x_train, y_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
train_loss_list = []
iters_num = 100
learning_rate = 1
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
for i in range(iters_num):
    grad = network.numerical_gradient(x_train, y_train)
    for key in {'W1', 'b1', 'W2', 'b2'}:
        network.params[key] -= learning_rate * grad[key]
        train_loss_list.append(loss)

ModuleNotFoundError: No module named 'numpy._core.numeric'

In [52]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(784, 50, 10)
iters_num = 1000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)
iter_per_epoch = 10
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    grad = network.gradient(x_batch, t_batch)
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))        

ModuleNotFoundError: No module named 'numpy._core.numeric'

### 오늘의 연습
### 최소 20개 이상의 자료에 대한 좌표를 지정.
### 또한 각 10개 이상의 동일 그룹에 대해 0, 1, ~ 등으로 출력 레이블을 지정
### 테스트 좌표는 4개 이상 지정

### 예시

In [11]:
x_train = np.array([[0, 0], #입력 4개->20개
                    [0, 1],
                    [1, 0],
                    [1, 1]])
# OR 게이트
t_train = np.array([[0],    #출력 레이블 2개(0, 1) -> ""
                    [1],
                    [1],
                    [1]])

x_test = np.array([[0.5, 0.5], [1.1, 0.9]]) #테스트 좌표 2개 -> 4개
t_test = np.array([[1], [1]])               #테스트 좌표의 레이블 2개 -> 2개

network = TwoLayerNet(2, 3, 1) #(784, 50, 10)-> (2, 5||3, 1)
iters_num = 10000                 #10000->100
train_size = x_train.shape[0]
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = 100
for i in range(iters_num):
    grad = network.gradient(x_train, t_train)
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    loss = network.loss(x_train, t_train)
    train_loss_list.append(loss)
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))        

train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train acc, test acc | 1.0, 1.0
train ac

# 과제

In [ ]:
import numpy as np
import random
from functions import *
from gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01): #입력 크기, 히든 레이어 크기, 출력 크기, 실수값으로 조지기
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) #입력x히든 사이즈로 난수 배열 만들고 실수로 값 변환
        self.params['b1'] = np.zeros(hidden_size)   #히든 레이어 개수만큼 
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        return y
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0]) #np.sum(y==t) y==t인 case의 횟수를 저장, 이를 평균을 내줌
        return accuracy                               #그렇게 구한 정확도를 반환
    
    def numerical_gradient(self, x, t):                 #손실함수를 w에 대해 미분함 -> gradient를 반환
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):   #x=20x2, t=20x1
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        batch_num = x.shape[0]  #20
        #forward
        a1 = np.dot(x, W1) + b1 
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        #backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)
        return grads


#학습데이터 20개 생성
x_train0 = np.random.uniform(0, 1, (10, 2))
x_train0 = np.round(x_train0, decimals=2)

x_train1 = np.random.uniform(1, 2, (10, 2))
x_train1 = np.round(x_train1, decimals=2)

x_train = np.vstack((x_train0, x_train1))

# 출력 레이블: 0 || 1
t_train = np.where(np.any(x_train > 1, axis=1), 1, 0).reshape(-1, 1)

# 테스트 데이터 4개
x_test = np.random.uniform(0, 2, (20, 2))
x_test = np.round(x_test, decimals=2)
# x_test의 예상 레이블 4개
t_test = np.where(np.any(x_test > 1, axis=1), 1, 0).reshape(-1, 1)     

network = TwoLayerNet(2, 3, 1) #(784, 50, 10)-> (2, 5||3, 1), (#입력, #히든 레이어, #출력)
iters_num = 3000                 #10000->100
train_size = x_train.shape[0] #20
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = 100
for i in range(iters_num):
    grad = network.gradient(x_train, t_train)
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    loss = network.loss(x_train, t_train)
    train_loss_list.append(loss)
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))   

In [ ]:
print(x_train)
print(x_test)

## 나도 이거 뭔지 기억 안 남

In [ ]:
# import numpy as np
# import random

# # 20개 이상의 학습 데이터를 생성 (10개씩 두 그룹으로 나눔)
# x_train = np.random.uniform(0, 2, (20, 2))
# x_train = np.round(x_train, decimals=2)

# # 출력 레이블 설정 (그룹 0은 0, 그룹 1은 1로 설정)
# t_train = np.where(np.any(x_train > 1, axis=1), 1, 0)   

# # 테스트 데이터 (4개 이상의 좌표)
# x_test = np.random.uniform(0, 2, (4, 2))
# x_test = np.round(x_test, decimals=2)

# # 테스트 데이터의 예상 레이블
# t_test = np.where(np.any(x_test > 1, axis=1), 1, 0)    


# print('x_train')
# print(x_train)

# print('\nt_train')
# print(t_train)

# print('\nx_test')
# print(x_test)

# print('\nt_test')
# print(t_test)  